# Ceneo Scraper

## Import bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup

## Strona produktu

In [ ]:
produkt_id = "84514582"
url = f"https://www.ceneo.pl/{produkt_id}#tab=reviews"

## Wysłanie żądania dostepu do zasobu

In [ ]:
response = requests.get(url)
response.status_code

In [ ]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")
opinion = page.select_one("div.js_product-review")
print(type(opinions))
print(type(opinion))
print(opinion)